**10. Regular Expression Matching**
 
**Hard**

**Companies:** Adobe Airbnb Alibaba Amazon Apple Bloomberg ByteDance Coursera Cruise Automation eBay Facebook Google Houzz Lyft Microsoft Oracle Palantir Technologies Pocket Gems Twitter Uber VMware Zulily

Given an input string s and a pattern p, implement regular expression matching with support for '.' and '*' where:

- '.' Matches any single character.​​​​
- '*' Matches zero or more of the preceding element.
The matching should cover the entire input string (not partial).

 

**Example 1:**
```python
Input: s = "aa", p = "a"
Output: false
```
**Explanation:** "a" does not match the entire string "aa".

**Example 2:**
```python
Input: s = "aa", p = "a*"
Output: true
```
**Explanation:** '*' means zero or more of the preceding element, 'a'. Therefore, by repeating 'a' once, it becomes "aa".

**Example 3:**
```python
Input: s = "ab", p = ".*"
Output: true
```
**Explanation:** ".*" means "zero or more (*) of any character (.)".
 

**Constraints:**

- 1 <= s.length <= 20
- 1 <= p.length <= 20
- s contains only lowercase English letters.
- p contains only lowercase English letters, '.', and '*'.
- It is guaranteed for each appearance of the character '*', there will be a previous valid character to match.
 


In [ ]:


# ==============================================================
# ✅ APPROACH 1 — RECURSION (BRUTE FORCE)
# ==============================================================
# ALGORITHM:
# 1️⃣ Compare pattern and string from left to right.
# 2️⃣ At each step:
#     - If pattern is empty → match if string is also empty.
#     - Check if the first characters match:
#          first_match = (s and (s[0] == p[0] or p[0] == '.'))
#     - If next char in pattern is '*':
#          Two possibilities:
#             (a) Ignore '*' and its preceding char → move 2 chars in p
#             (b) Use '*' if first chars match → move 1 char in s
#     - Else:
#          Continue matching next characters if first_match.
#
# TIME:  O(2^(m+n)) — exponential recursion
# SPACE: O(m+n)
# ==============================================================

class Solution_Recursive:
    def isMatch(self, s: str, p: str) -> bool:
        if not p:
            return not s

        first_match = bool(s) and (s[0] == p[0] or p[0] == '.')

        if len(p) >= 2 and p[1] == '*':
            # Case 1: Skip '*' pattern
            # Case 2: Consume one char from s if first matches
            return (self.isMatch(s, p[2:]) or
                    (first_match and self.isMatch(s[1:], p)))
        else:
            return first_match and self.isMatch(s[1:], p[1:])


In [ ]:


# ==============================================================
# ✅ APPROACH 2 — DP (TOP-DOWN with Memoization)
# ==============================================================
# ALGORITHM:
# 1️⃣ Use recursion with memo to avoid recomputing subproblems.
# 2️⃣ Define dp(i, j): True if s[i:] matches p[j:].
# 3️⃣ Recurrence:
#      - Base: if j == len(p): return i == len(s)
#      - first_match = i < len(s) and (s[i] == p[j] or p[j] == '.')
#      - If '*' follows p[j]:
#           dp(i, j) = dp(i, j+2) or (first_match and dp(i+1, j))
#        Else:
#           dp(i, j) = first_match and dp(i+1, j+1)
#
# TIME:  O(m*n)
# SPACE: O(m*n)
# ==============================================================

from functools import lru_cache

class Solution_TopDownDP:
    def isMatch(self, s: str, p: str) -> bool:
        @lru_cache(None)
        def dp(i, j):
            if j == len(p):
                return i == len(s)

            first_match = i < len(s) and (s[i] == p[j] or p[j] == '.')

            if j + 1 < len(p) and p[j + 1] == '*':
                # Skip '*' (zero occurrence) OR
                # Use '*' (consume one char if match)
                return dp(i, j + 2) or (first_match and dp(i + 1, j))
            else:
                return first_match and dp(i + 1, j + 1)

        return dp(0, 0)



In [ ]:

# ==============================================================
# ✅ APPROACH 3 — DP (BOTTOM-UP)
# ==============================================================
# ALGORITHM:
# 1️⃣ Define dp[i][j] → True if s[:i] matches p[:j].
# 2️⃣ Initialize:
#      dp[0][0] = True (empty string matches empty pattern)
#      dp[0][j] = True if p[:j] can match empty string (like a*, a*b*, etc.)
# 3️⃣ Transition:
#     - If p[j-1] != '*':
#           dp[i][j] = dp[i-1][j-1] and (s[i-1] == p[j-1] or p[j-1] == '.')
#     - If p[j-1] == '*':
#           dp[i][j] = dp[i][j-2]  # zero occurrence
#                     or (dp[i-1][j] and (s[i-1] == p[j-2] or p[j-2] == '.'))
# 4️⃣ Answer: dp[len(s)][len(p)]
#
# TIME:  O(m*n)
# SPACE: O(m*n)
# ==============================================================

class Solution_BottomUpDP:
    def isMatch(self, s: str, p: str) -> bool:
        m, n = len(s), len(p)
        dp = [[False] * (n + 1) for _ in range(m + 1)]
        dp[0][0] = True

        # Handle patterns like a*, a*b*, a*b*c*
        for j in range(2, n + 1):
            if p[j - 1] == '*':
                dp[0][j] = dp[0][j - 2]

        # Fill DP table
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if p[j - 1] == '*':
                    dp[i][j] = dp[i][j - 2] or (
                        dp[i - 1][j] and (s[i - 1] == p[j - 2] or p[j - 2] == '.')
                    )
                else:
                    dp[i][j] = dp[i - 1][j - 1] and (
                        s[i - 1] == p[j - 1] or p[j - 1] == '.'
                    )

        return dp[m][n]
